# 从键盘收集示范数据

**中文教程：如何通过键盘操作收集机器人示范数据**

本教程将指导您如何为给定环境收集示范数据。
任务是抓取一个杯子并将其放在盘子上。当杯子放在盘子上、夹爪打开、末端执行器位于杯子上方时，环境会识别为任务成功。

<img src="./media/teleop.gif" width="480" height="360">

## 键盘控制说明
- **WASD键**：控制XY平面移动
- **RF键**：控制Z轴（上下）移动
- **QE键**：控制倾斜角度
- **方向键**：控制其他旋转
- **空格键**：切换夹爪状态（开/关）
- **Z键**：重置环境并丢弃当前回合数据

## 屏幕显示说明
叠加的图像显示：
- **右上角**：机器人视角
- **右下角**：第一人称视角
- **左上角**：左侧视角
- **左下角**：俯视图


In [14]:
# Cell 1 - 设置环境变量（必须第一个运行）
import os

# 1. 设置DISPLAY
os.environ['DISPLAY'] = ':0'
os.environ['XAUTHORITY'] = os.path.expanduser('~/.Xauthority')
print(f"✓ DISPLAY设置为: {os.environ['DISPLAY']}")

# 2. 强制使用GPU渲染（关键！）
os.environ['MUJOCO_GL'] = 'egl'  # EGL后端GPU加速
print(f"✓ MUJOCO_GL: egl (GPU硬件加速)")

# 3. NVIDIA GPU优化
os.environ['__GL_SYNC_TO_VBLANK'] = '0'  # 关闭垂直同步
os.environ['__GL_YIELD'] = 'NOTHING'      # 减少CPU等待
print("✓ NVIDIA GPU优化已启用")

# 4. OpenGL性能优化
os.environ['__GL_FSAA_MODE'] = '0'        # 关闭抗锯齿
os.environ['__GL_LOG_MAX_ANISO'] = '0'    # 关闭各向异性过滤
print("✓ OpenGL性能优化已启用")

✓ DISPLAY设置为: :0
✓ MUJOCO_GL: egl (GPU硬件加速)
✓ NVIDIA GPU优化已启用
✓ OpenGL性能优化已启用


In [15]:
# 导入必要的库
import sys          # 系统相关功能
import random       # 随机数生成
import numpy as np  # 数值计算库
import os           # 操作系统接口
from PIL import Image  # 图像处理库
from mujoco_env.y_env import SimpleEnv  # 导入MuJoCo环境接口
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset  # 导入数据集处理库

In [16]:
# 设置随机种子
# 如果要随机化物体位置，将此设置为None
# 如果固定种子，每次运行时物体位置都会相同
SEED = 0 
# SEED = None <- 取消此行注释以随机化物体位置
REPO_NAME = 'omy_pnp'
NUM_DEMO = 1 # Number of demonstrations to collect
ROOT = "./demo_data" # The root directory to save the demonstrations


In [17]:
# 加载MuJoCo环境
TASK_NAME = 'Put mug cup on the plate' 
xml_path = './asset/example_scene_y.xml'  # 场景描述文件，定义了机器人、物体和环境的物理属性

# 创建环境实例
# action_type="ee_pose"表示控制方式为末端执行器位姿控制（而非关节角度控制）
#env = SimpleEnv(xml_path, action_type="ee_pose")
PnPEnv = SimpleEnv(xml_path, seed = SEED, state_type = 'joint_angle')
# 重置环境，使用指定的随机种子
#env.reset(seed=SEED)



-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[24] n_qvel:[22] n_qacc:[22] n_ctrl:[10]
 integrator:[IMPLICITFAST]

n_body:[21]
 [0/21] [world] mass:[0.00]kg
 [1/21] [front_object_table] mass:[1.00]kg
 [2/21] [camera] mass:[0.00]kg
 [3/21] [camera2] mass:[0.00]kg
 [4/21] [camera3] mass:[0.00]kg
 [5/21] [link1] mass:[2.06]kg
 [6/21] [link2] mass:[3.68]kg
 [7/21] [link3] mass:[2.39]kg
 [8/21] [link4] mass:[1.40]kg
 [9/21] [link5] mass:[1.40]kg
 [10/21] [link6] mass:[0.65]kg
 [11/21] [camera_center] mass:[0.00]kg
 [12/21] [tcp_link] mass:[0.32]kg
 [13/21] [rh_p12_rn_r1] mass:[0.07]kg
 [14/21] [rh_p12_rn_r2] mass:[0.02]kg
 [15/21] [rh_p12_rn_l1] mass:[0.07]kg
 [16/21] [rh_p12_rn_l2] mass:[0.02]kg
 [17/21] [body_obj_mug_5] mass:[0.00]kg
 [18/21] [object_mug_5] mass:[0.08]kg
 [19/21] [body_obj_plate_11] mass:[0.00]kg
 [20/21] [object_plate_11] mass:[0.10]kg
body_total_mass:[13.27]kg

n_geom:[83]
geom_names:['floor', 

## Define Dataset Fatures and Create your dataset!
The dataset is contained as follows:
```
fps = 20,
features={
    "observation.image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channels"],
    },
    "observation.wrist_image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channel"],
    },
    "observation.state": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["state"], # x, y, z, roll, pitch, yaw
    },
    "action": {
        "dtype": "float32",
        "shape": (7,),
        "names": ["action"], # 6 joint angles and 1 gripper
    },
    "obj_init": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["obj_init"], # just the initial position of the object. Not used in training.
    },
},
```


This will make the dataset on './demo_data' folder, which will look like this,

```
.
├── data
│   ├── chunk-000
│   │   ├── episode_000000.parquet
│   │   └── ...
├── meta
│   ├── episodes.jsonl
│   ├── info.json
│   ├── stats.json
│   └── tasks.jsonl
└── 
```


In [18]:
create_new = True
if os.path.exists(ROOT):
    print(f"Directory {ROOT} already exists.")
    ans = input("Do you want to delete it? (y/n) ")
    if ans == 'y':
        import shutil
        shutil.rmtree(ROOT)
    else:
        create_new = False


if create_new:
    dataset = LeRobotDataset.create(
                repo_id=REPO_NAME,
                root = ROOT, 
                robot_type="omy",
                fps=20, # 20 frames per second
                features={
                    "observation.image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channels"],
                    },
                    "observation.wrist_image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channel"],
                    },
                    "observation.state": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["state"], # x, y, z, roll, pitch, yaw
                    },
                    "action": {
                        "dtype": "float32",
                        "shape": (7,),
                        "names": ["action"], # 6 joint angles and 1 gripper
                    },
                    "obj_init": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["obj_init"], # just the initial position of the object. Not used in training.
                    },
                },
                image_writer_threads=10,
                image_writer_processes=5,
        )
else:
    print("Load from previous dataset")
    dataset = LeRobotDataset(REPO_NAME, root=ROOT)

Directory ./demo_data already exists.


## Keyboard Control
You can teleop your robot with keyboard and collect dataset
```
---------     -----------------------
   w       ->        backward
s  a  d        left   forward   right
---------      -----------------------
In x, y plane

---------
R: Moving Up
F: Moving Down
---------
In z axis

---------
Q: Tilt left
E: Tilt right
UP: Look Upward
Down: Look Donward
Right: Turn right
Left: Turn left
---------
For rotation

---------
SPACEBAR: Toggle Gripper
--------

---------
z: reset
--------
```
Reseting your environment will remove the cache data of the current demonstration and restart collection.

### Now let's teleop our robot and collect data!

**To receive the success signal, you have to release the gripper and move upwards above the mug!**

In [19]:
action = np.zeros(7)
episode_id = 0
record_flag = False # Start recording when the robot starts moving
while PnPEnv.env.is_viewer_alive() and episode_id < NUM_DEMO:
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # check if the episode is done
        done = PnPEnv.check_success()
        if done: 
            # Save the episode data and reset the environment
            dataset.save_episode()
            PnPEnv.reset(seed = SEED)
            episode_id += 1
        # Teleoperate the robot and get delta end-effector pose with gripper
        action, reset  = PnPEnv.teleop_robot()
        if not record_flag and sum(action) != 0:
            record_flag = True
            print("Start recording")
        if reset:
            # Reset the environment and clear the episode buffer
            # This can be done by pressing 'z' key
            PnPEnv.reset(seed=SEED)
            # PnPEnv.reset()
            dataset.clear_episode_buffer()
            record_flag = False
        # Step the environment
        # Get the end-effector pose and images
        ee_pose = PnPEnv.get_ee_pose()
        agent_image,wrist_image = PnPEnv.grab_image()
        # # resize to 256x256
        agent_image = Image.fromarray(agent_image)
        wrist_image = Image.fromarray(wrist_image)
        agent_image = agent_image.resize((256, 256))
        wrist_image = wrist_image.resize((256, 256))
        agent_image = np.array(agent_image)
        wrist_image = np.array(wrist_image)
        joint_q = PnPEnv.step(action)
        if record_flag:
            # Add the frame to the dataset
            dataset.add_frame( {
                    "observation.image": agent_image,
                    "observation.wrist_image": wrist_image,
                    "observation.state": ee_pose, 
                    "action": joint_q,
                    "obj_init": PnPEnv.obj_init_pose,
                    # "task": TASK_NAME,
                }, task = TASK_NAME
            )
        PnPEnv.render(teleop=True)

Start recording
DONE INITIALIZATION
Start recording


Creating parquet from Arrow format: 100%|██████████| 9/9 [00:00<00:00, 58.91ba/s]


DONE INITIALIZATION


In [20]:
PnPEnv.env.close_viewer()

In [21]:
# Clean up the images folder
import shutil
shutil.rmtree(dataset.root / 'images')